In [ ]:
# Importing All Necessary Libraries

import os
import numpy as np
import cv2

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Importing the Dataset

# Define the paths to the train and test folders
train_normal_path = '/content/drive/MyDrive/PneumoniaCTData/chest_xray/train/NORMAL'
train_pneumonia_path = '/content/drive/MyDrive/PneumoniaCTData/chest_xray/train/PNEUMONIA'
test_normal_path = '/content/drive/MyDrive/PneumoniaCTData/chest_xray/test/NORMAL'
test_pneumonia_path = '/content/drive/MyDrive/PneumoniaCTData/chest_xray/test/PNEUMONIA'

In [ ]:
def preprocess_images(image_paths):
    preprocessed_images = []
    for path in image_paths:
        # Load the image without resizing
        image = cv2.imread(path)
        preprocessed_images.append(image)
    return np.array(preprocessed_images)

In [ ]:
# Get the list of image paths for training and testing
train_normal_images = [os.path.join(train_normal_path, image) for image in os.listdir(train_normal_path)]
train_pneumonia_images = [os.path.join(train_pneumonia_path, image) for image in os.listdir(train_pneumonia_path)]
test_normal_images = [os.path.join(test_normal_path, image) for image in os.listdir(test_normal_path)]
test_pneumonia_images = [os.path.join(test_pneumonia_path, image) for image in os.listdir(test_pneumonia_path)]

# Preprocess the images
train_normal_data = preprocess_images(train_normal_images)
train_pneumonia_data = preprocess_images(train_pneumonia_images)
test_normal_data = preprocess_images(test_normal_images)
test_pneumonia_data = preprocess_images(test_pneumonia_images)

<ipython-input-3-c66f91b383da>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(preprocessed_images)


In [ ]:
# Create labels for the data
train_normal_labels = np.zeros(len(train_normal_data))
train_pneumonia_labels = np.ones(len(train_pneumonia_data))
test_normal_labels = np.zeros(len(test_normal_data))
test_pneumonia_labels = np.ones(len(test_pneumonia_data))

# Concatenate the data and labels
X_train = np.concatenate((train_normal_data, train_pneumonia_data), axis=0)
y_train = np.concatenate((train_normal_labels, train_pneumonia_labels), axis=0)
X_test = np.concatenate((test_normal_data, test_pneumonia_data), axis=0)
y_test = np.concatenate((test_normal_labels, test_pneumonia_labels), axis=0)

In [ ]:
# Resize data for deep learning
X_train= np.array(X_train).reshape(-1, 200, 200, 1)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, 200, 200, 1)
y_test = np.array(y_test)

In [ ]:
# Data Augmentation
data_augmenter = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        rotation_range=45,
        zoom_range = 0.2,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

data_augmenter.fit(X_train)

In [ ]:
# Reshape X_train and X_test to have 2 dimensions
num_train_samples = X_train.shape[0]
num_test_samples = X_test.shape[0]
X_train = X_train.reshape(num_train_samples, -1)
X_test = X_test.reshape(num_test_samples, -1)

In [ ]:
# Standardize Data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train and evaluate K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_f1 = f1_score(y_test, knn_predictions)
knn_confusion_matrix = confusion_matrix(y_test, knn_predictions)

# Print the evaluation results for KNN
print("KNN Results:")
print("Accuracy:", knn_accuracy)
print("F1 Score:", knn_f1)
print("Confusion Matrix:")
print(knn_confusion_matrix)
print()

KNN Results:
Accuracy: 0.7756410256410257
F1 Score: 0.8461538461538463
Confusion Matrix:
[[ 99 135]
 [  5 385]]



In [ ]:
# Train and evaluate Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
logreg_predictions = logreg.predict(X_test)
logreg_accuracy = accuracy_score(y_test, logreg_predictions)
logreg_f1 = f1_score(y_test, logreg_predictions)
logreg_confusion_matrix = confusion_matrix(y_test, logreg_predictions)

# Print the evaluation results for Logistic Regression
print("Logistic Regression Results:")
print("Accuracy:", logreg_accuracy)
print("F1 Score:", logreg_f1)
print("Confusion Matrix:")
print(logreg_confusion_matrix)
print()

Logistic Regression Results:
Accuracy: 0.7419871794871795
F1 Score: 0.8270676691729324
Confusion Matrix:
[[ 78 156]
 [  5 385]]



In [ ]:
# Reshape X_train and X_test
X_train = np.reshape(X_train, (-1, 200, 200, 1))
X_test = np.reshape(X_test, (-1, 200, 200, 1))

# Train and evaluate CNN

# CNN model architecture
model = Sequential()
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=X_train.shape[1:], padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32)

# Evaluate the model on the test set
cnn_predictions = model.predict(X_test)
cnn_predictions = np.round(cnn_predictions).flatten()

# Calculate evaluation metrics
cnn_accuracy = accuracy_score(y_test, cnn_predictions)
cnn_f1 = f1_score(y_test, cnn_predictions)
cnn_confusion_matrix = confusion_matrix(y_test, cnn_predictions)

# Print the evaluation results for CNN
print("CNN Results:")
print("Accuracy:", cnn_accuracy)
print("F1 Score:", cnn_f1)
print("Confusion Matrix:")
print(cnn_confusion_matrix)
print()

Epoch 1/3
163/163 [==============================] - 2557s 16s/step - loss: 3.2185 - accuracy: 0.8861
Epoch 2/3
163/163 [==============================] - 2548s 16s/step - loss: 0.0927 - accuracy: 0.9666
Epoch 3/3
20/20 [==============================] - 77s 4s/step
CNN Results:
Accuracy: 0.7227564102564102
F1 Score: 0.8177028451001054
Confusion Matrix:
[[ 63 171]
 [  2 388]]

